# Python unittest framework and mock

# TODO: Order of patched args.

To run tests in the notebook environment, we need a utility method:

In [ ]:
def run_tests(test_cls):
    tests = unittest.TestLoader().loadTestsFromTestCase(test_cls)
    unittest.TextTestRunner(verbosity=2).run(tests)

## unittest.TestCase

In [ ]:
import unittest

class MyTest(unittest.TestCase):

    def test_sanity(self):
        """Verify sanity.
        """
        self.assertTrue(True)

run_tests(MyTest)

## Recipe

In [ ]:
class Recipe(object):
    """For now, just an encapsulation of ingredients.
    """
    def __init__(self, ingredients=None):
        """
        :type ingredients: [:py:class:`Ingredient`, ...]
        :arg ingredients: list of recipe ingredients
        """
        self.ingredients = ingredients

## RecipeTests 

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_init(self):
        """Verify member data initialization.
        """
        test_ingredients = [Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)
        self.assertEqual(test_ingredients, test_recipe.ingredients)
        
run_tests(RecipeTests)

## No Ingredient class? No problem. Introducing Mock.

In [ ]:
import mock

class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_init(self):
        """Verify member data initialization.
        """
        test_ingredients = mock.Mock(name='ingredients')
        test_recipe = Recipe(ingredients=test_ingredients)
        self.assertEqual(test_ingredients, test_recipe.ingredients)
        
run_tests(RecipeTests)

## <span style="color:#f0f">Best practice: Give your Mock objects meaningful names.</span>

## Recipe.is_vegetarian

In [ ]:
class Recipe(object):
    """For now, just an encapsulation of ingredients.
    """
    def __init__(self, ingredients=None):
        self.ingredients = ingredients
        
    def is_vegetarian(self):
        """Recipe is vegetarian iff all ingredients are vegetarian.
        
        :rtype: bool
        :return: True if recipe is vegetarian, False otherwise
        
        """
        if not self.ingredients:
            raise ValueError("Recipe has no ingredients")
        for ingredient in self.ingredients:
            if not ingredient.is_vegetarian():
                return False
        return True

## RecipeTests.test_is_vegetarian

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_is_vegetarian(self):
        """Verify calls made by is_vegetarian.
        """
        test_ingredients = mock.Mock(name='ingredients')
        test_recipe = Recipe(ingredients=test_ingredients)
        test_recipe.is_vegetarian()
        
run_tests(RecipeTests)

## Mock object not iterable? No problem. Introducing MagicMock.

**Try it:** Replace Mock with MagicMock.

## Best practice: Be careful with MagicMock.

## Alternatively, use list of Mocks

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_is_vegetarian(self):
        """Verify calls made by is_vegetarian.
        """
        test_ingredients = [mock.Mock(name='ingredient_1'), mock.Mock(name='ingredient_2')
        test_recipe = Recipe(ingredients=test_ingredients)
        test_recipe.is_vegetarian()
        
run_tests(RecipeTests)

## Ingredient class

In [ ]:
class Ingredient(object):
    """Not very interesting yet. 
    """     
    def is_veggie(self):
        """Intentionally misnamed method.
        """
        raise NotImplementedError()

**Try it:** Re-run original RecipeTests.test_init, now that Ingredient is defined.

## Mock.spec_set

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_is_vegetarian(self):
        """Verify calls made by is_vegetarian.
        """
        test_ingredients = [mock.Mock(name='ingredient_1',
                                     spec_set=Ingredient), 
                            mock.Mock(name='ingredient_2',
                                     spec_set=Ingredient)]
        test_recipe = Recipe(ingredients=test_ingredients)
        test_recipe.is_vegetarian()
        
run_tests(RecipeTests)

## Best practice: Prefer spec_set to autospec

**Try it:** Replace spec_set with autospec.

## Fix Ingredient.is_vegetarian

In [ ]:
class Ingredient(object):
    """Not very interesting yet. 
    """     
    def is_vegetarian(self):
        raise NotImplementedError()

## RecipeTests.test_is_vegetarian with Ingredient objects

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_is_vegetarian(self):
        """Verify calls made by is_vegetarian.
        """
        test_ingredients = [Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)
        test_recipe.is_vegetarian()
        
run_tests(RecipeTests)

## Not ready to implement Ingredient.is_vegetarian? No problem. Introducing mock.patch.object.

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian(self, is_ingredient_vegetarian):
        """Verify calls made by is_vegetarian.
        """
        # Set up test data
        test_ingredients = [Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)
        
        # Make call
        test_recipe.is_vegetarian()
        
        # Verify mocks
        self.assertEqual(2, is_ingredient_vegetarian.call_count)
        
run_tests(RecipeTests)

## Best practice: Prefer mock.patch.object to mock.patch.

## Mock.return_value and Mock.call_count

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian__ingredients_vegetarian(self, is_veg_ingredient):
        """Verify return value of is_vegetarian when ingredients are vegetarian.
        """
        # Set up mocks and test data 
        is_veg_ingredient.return_value = True
        test_ingredients = [Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)
        
        # Make call
        result = test_recipe.is_vegetarian()
        
        # Verify result
        self.assertTrue(result)
        
        # Verify mocks
        self.assertEqual(len(test_ingredients), is_veg_ingredient.call_count)
        
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian__ingredients_non_vegetarian(self, is_veg_ingredient):
        """Verify return value of is_vegetarian when ingredients are not vegetarian.
        """
        pass
        
run_tests(RecipeTests)

**Try it:** Implement test_is_vegetarian__ingredients_non_vegetarian.

## Mock.side_effect

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian__some_ingredients_vegetarian(self, is_veg_ingredient):
        """Verify return value of is_vegetarian when ingredients are vegetarian.
        """
        # Set up mocks and test data
        is_veg_ingredient.side_effect = [True, False, True]
        test_ingredients = [Ingredient(), Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)
        
        # Make call
        result = test_recipe.is_vegetarian()
        
        # Verify result
        self.assertFalse(result)
        
        # Verify mocks
        # Stop calling ingredient.is_vegetarian after first False return value
        self.assertEqual(2, is_veg_ingredient.call_count)
        
run_tests(RecipeTests)

## Recipe and Ingredient is_vegetarian methods support _strict_ keyword argument

In [ ]:
class Recipe(object):
    """For now, just an encapsulation of ingredients.
    """
    def __init__(self, ingredients=None):
        self.ingredients = ingredients
        
    def is_vegetarian(self, strict=True):
        """Recipe is vegetarian iff all ingredients are vegetarian.
        
        :rtype: bool
        :return: True if recipe is vegetarian, False otherwise
        
        :type strict: bool
        :arg strict: True if only strictly vegetarian recipes should pass
        
        """
        if not self.ingredients:
            raise ValueError("Recipe has no ingredients")
        for ingredient in self.ingredients:
            if not ingredient.is_vegetarian(strict=strict):
                return False
        return True

class Ingredient(object):
    """Not very interesting yet. 
    """     
    def is_vegetarian(self, strict=True):
        raise NotImplementedError()

## Mock.assert_called_once_with

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian__strict__one_ingredient(self, is_veg_ingredient):
        """Verify calls made by is_vegetarian.
        """
        # Set up mocks and test data
        is_veg_ingredient.return_value = True
        mock_strict_arg = mock.Mock(name='strict_arg')
        test_recipe = Recipe(ingredients=[Ingredient()])

        # Make call
        test_recipe.is_vegetarian(strict=mock_strict_arg)
        
        # Verify mocks
        is_veg_ingredient.assert_called_once_with(strict=mock_strict_arg)
       
run_tests(RecipeTests)

## Best practice: Prefer assert_called_once_with to assertEqual(1, my_mock.call_count)

## Mock.call_args_list and Mock.call

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    @mock.patch.object(Ingredient, 'is_vegetarian')
    def test_is_vegetarian__strict__multiple_ingredients(self, is_veg_ingredient):
        """Verify calls made by is_vegetarian.
        """
        # Set up mocks and test data
        is_veg_ingredient.return_value = True
        mock_strict_arg = mock.Mock(name='strict_arg')
        test_ingredients = [Ingredient(), Ingredient()]
        test_recipe = Recipe(ingredients=test_ingredients)

        # Make call
        test_recipe.is_vegetarian(strict=mock_strict_arg)
        
        # Verify mocks
        mock_calls = is_veg_ingredient.call_args_list
        self.assertEqual(len(test_ingredients), len(mock_calls))
        for mock_call in mock_calls:
            self.assertEqual(mock.call(strict=mock_strict_arg), mock_call)
        
run_tests(RecipeTests)

## unittest.TestCase.assertRaisesRegexp

In [ ]:
class RecipeTests(unittest.TestCase):
    """Verify Recipe behavior.
    """
    def test_is_vegetarian__no_ingredients(self):
        """Verify ValueError if no ingredients are set on recipe.
        """
        # Set up mocks and test data
        for no_ingredients in (None, []):
            test_recipe = Recipe(ingredients=no_ingredients)

            # Make call
            self.assertRaisesRegexp(ValueError,
                                   'Recipe has no ingredients',
                                    test_recipe.is_vegetarian)
        
        
run_tests(RecipeTests)

## Best practice: Prefer assertRaisesRegexp to assertRaises